In [53]:

import random
import pandas as pd
import numpy as np
import os
import patsy

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings(action='ignore') 


In [67]:
train = pd.read_csv('train.csv')
info = pd.read_csv('building_info.csv')
test = pd.read_csv('test.csv')

train_info = pd.merge(train,info,how='outer',on='건물번호')

train_info0 = train_info.fillna(0).replace('-',0)
train_info1 = train_info0.replace('건물기타',1).replace('공공',2).replace('대학교',3).replace('백화점및아울렛',4).replace('병원',5).replace('상용',6).replace('아파트',7).replace('연구소',8).replace('지식산업센터',9).replace('할인마트',10).replace('호텔및리조트',11).replace('데이터센터',12)

train_info1['태양광용량(kW)'] = pd.to_numeric(train_info1['태양광용량(kW)'])
train_info1['ESS저장용량(kWh)'] = pd.to_numeric(train_info1['ESS저장용량(kWh)'])
train_info1['PCS용량(kW)'] = pd.to_numeric(train_info1['PCS용량(kW)'])
train_info1 = train_info1.drop(['num_date_time'], axis = 1)
train_info1['sequence'] = train_info1.groupby('건물번호').cumcount()+1
train_info1 = train_info1.drop(['일시'], axis = 1)


encoder = OneHotEncoder()
encoder.fit(train_info1[['건물유형']])
onehot = encoder.transform(train_info1[['건물유형']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names_out()
train_info1 = pd.concat([train_info1,onehot],axis = 1)
train_info1 = train_info1.drop(columns = ['건물유형'])

# encoder = OneHotEncoder()
# encoder.fit(train_info1[['건물번호']])
# onehot = encoder.transform(train_info1[['건물번호']])
# onehot = onehot.toarray()
# onehot = pd.DataFrame(onehot)
# onehot.columns = encoder.get_feature_names_out()
# train_info1 = pd.concat([train_info1,onehot],axis = 1)
# train_info1 = train_info1.drop(columns = ['건물번호'])


In [68]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train_x = train_info1.drop(columns = ['전력소비량(kWh)'])
train_y = train_info1['전력소비량(kWh)']
model = RandomForestRegressor()
model.fit(train_x, train_y)



RandomForestRegressor()

In [72]:
test_x = test.drop(columns = ['num_date_time','일시'])


#preds = model.predict(test_x)